In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('SolarDataProcessed.csv',index_col=0)
df.head()

,PRECTOT,QV2M,PS,TS,KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN
0,0.0,0.003886,99.11,11.85,0.68,3.97,6.50
1,0.0,0.003804,99.05,11.48,0.68,3.97,6.50
2,0.0,0.003462,99.13,11.86,0.76,4.42,6.36
3,0.0,0.003457,99.06,12.04,0.76,4.42,6.36
4,0.0,0.002913,99.31,10.09,0.73,4.29,6.50


In [2]:
'''from sklearn import preprocessing
from sklearn.model_selection import train_test_split
x = df[df.columns[:10]]
y = df['ALLSKY_SFC_SW_DWN']
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)
#Select numerical columns which needs to be normalized
train_norm = x_train[x_train.columns[0:10]]
test_norm = x_test[x_test.columns[0:10]]
# Normalize Training Data 
std_scale = preprocessing.StandardScaler().fit(train_norm)
x_train_norm = std_scale.transform(train_norm)
#Converting numpy array to dataframe
training_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) 
x_train.update(training_norm_col)
#x_train.head()
# Normalize Testing Data by using mean and SD of training set
x_test_norm = std_scale.transform(test_norm)
testing_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) 
x_test.update(testing_norm_col)'''

"from sklearn import preprocessing\nfrom sklearn.model_selection import train_test_split\nx = df[df.columns[:10]]\ny = df['ALLSKY_SFC_SW_DWN']\nx_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)\n#Select numerical columns which needs to be normalized\ntrain_norm = x_train[x_train.columns[0:10]]\ntest_norm = x_test[x_test.columns[0:10]]\n# Normalize Training Data \nstd_scale = preprocessing.StandardScaler().fit(train_norm)\nx_train_norm = std_scale.transform(train_norm)\n#Converting numpy array to dataframe\ntraining_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) \nx_train.update(training_norm_col)\n#x_train.head()\n# Normalize Testing Data by using mean and SD of training set\nx_test_norm = std_scale.transform(test_norm)\ntesting_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) \nx_test.update(testing_norm_col)"

In [3]:
from sklearn.model_selection import train_test_split
x = df[df.columns[:6]]
y = df['ALLSKY_SFC_SW_DWN']
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.8, random_state =  90)

In [4]:
# IMPORT ML CLASSIFIERS
from sklearn.linear_model import LinearRegression # Linear regression
from sklearn.ensemble import RandomForestRegressor # random forest regression
from sklearn.neural_network import MLPRegressor # neural network regression
from sklearn.svm import SVR # support vector regression

from sklearn import preprocessing # ML tools
from sklearn.model_selection import train_test_split # split data


from bokeh.plotting import figure, show, output_notebook

def plot_test(clf,X_test,y_test):
    y_predicted = clf.predict(X_test)

    p = figure(tools='pan,box_zoom,reset',x_range=[0, 100], title='Predicted vs Actual',y_axis_label='radiation')
    p.grid.minor_grid_line_color = '#eeeeee'

    p.line(range(len(y_test)),y_test,legend='actual',line_color='blue',line_width=2)
    p.line(range(len(y_test)),y_predicted,legend='prediction',line_color='red',line_width=2)
    output_notebook()
    show(p)
    return

'''def plot_real(clf,x,y_actual,index):
     Plot predictions for actual measurements.
    inputs:
        clf         as classifier   the trained algorithm
        x           as array        timeseries of measurement inputs
        y_actual    as array        corresponding timeseries of actual results
    
    y_predicted = clf.predict(x)

    p = figure(toolbar_location='right', title='Predicted vs Actual',y_axis_label='radiation',x_axis_type="datetime")
    p.grid.minor_grid_line_color = '#eeeeee'

    p.line(index,y_actual,legend='actual',line_color='blue')
    p.line(index,y_predicted,legend='prediction',line_color='red')
    output_notebook()
    show(p)
    return'''

def train_model(x_train, x_test, y_train, y_test,clf,debug=False):
    ''' Train algorithm.
    inputs:
        x       as array        features
        y       as array        label(s)
        clf     as scikit-learn classifier (untrained)
    returns:
        clf     as trained classifier
        accuracy  as float
    '''
    model = clf.fit(x_train,y_train)
    accuracy = clf.score(x_test,y_test)
    return clf, model, accuracy,x_test, y_test

def go(x_train, x_test, y_train, y_test,algorithm,debug=True):
    ''' Easy model train and test. '''
    clf, model, accuracy, x_test, y_test=train_model(x_train, x_test, y_train, y_test,algorithm,debug=True)
    print('Accuracy: %s percent'%str(accuracy*100))

    if debug:
        plot_test(clf,x_test,y_test)
        #plot_real(clf,x,y,df.index.values)
    return

'''def optimize_randomforest(x,y,try_n=10,try_f='auto',try_s=1):
     Find best combo of tunable params for random forest regressor.
    best_score = float('-inf') # initialize score
    for n in try_n:
        for f in try_f:
            for s in try_s:
                clf = RandomForestRegressor(oob_score=True,n_estimators=n,max_features=f,min_samples_leaf=s,n_jobs=-1)
                clf.fit(x,y)
                if clf.oob_score_ > best_score:
                    best_score, best_clf, best_n, best_f, best_s = clf.oob_score_, clf, n, f, s
    return clf, best_n, best_f, best_s

n=[100,200,300,500]
f=[2,4,6]
s=[1,2,4,8,16]
clf, n, f, s = optimize_randomforest(x_train,y_train,try_n=n,try_f=f,try_s=s)'''
#print('n_estimators: '+str(n))
#print('max_features: '+str(f))
#print('min_samples_leaf: '+str(s))
#go(x_train, x_test, y_train, y_test,RandomForestRegressor(n_estimators=n,max_features=f,min_samples_leaf=s,n_jobs=-1))
#go(x_train, x_test, y_train, y_test,LinearRegression())
go(x_train, x_test, y_train, y_test,SVR())

c:\python37\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy: 89.99053057530176 percent


Loading BokehJS ...

In [3]:
df.drop(['MO','DY'], axis = 1, inplace =True)